In [11]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

driver = webdriver.Chrome(ChromeDriverManager().install())

tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [12]:
driver.get("https://www.google.com.br/")

produto = 'iphone 12 64gb'
termos_banidos = 'mini watch'
preco_minimo = 3000
preco_maximo = 3500

driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

elementos = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')

for elemento in elementos:
    if "Shopping" in elemento.text:
        elemento.click()
        break

lista_resultados = driver.find_elements(By.CLASS_NAME, 'i0X6df')

for resultado in lista_resultados:
    preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
    nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text

    child = resultado.find_element(By.CLASS_NAME, 'XrAfOe')
    parent = child.find_element(By.XPATH, '..')
    link = parent.get_attribute('href')
    print(preco, nome, link)


  https://www.google.com.br/url?url=https://www.amazon.com.br/Apple-iPhone-12-64-GB-Branco/dp/B09B7X1814%3Fsource%3Dps-sl-shoppingads-lpcontext%26ref_%3Dfplfs%26psc%3D1%26smid%3DA1ZZFT5FULY4LN&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwi_-dSOvuH8AhVBqpUCHT32Av8QguUECMkL&usg=AOvVaw3ZCNjrmRTQbjVyO0ceSMXs
R$ 4.684,06 iPhone 12 Mini 64 Gb - Preto Apple https://www.google.com.br/url?url=https://www.ibyte.com.br/iphone-12-mini-apple-preto-64gb-desbloqueado-mgdx3bz-a/p&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwi_-dSOvuH8AhVBqpUCHT32Av8QguUECOEL&usg=AOvVaw0pUhtpXwaar85stbJvpzbc
R$ 3.799,05 Vitrine: iPhone 12 Apple 64GB Azul Tela 6,1 Câmera Traseira Dupla 12MP https://www.google.com.br/url?url=https://www.carrefour.com.br/vitrine-iphone-12-apple-64gb-azul-tela-61-camera-traseira-dupla-12mp-mp911874363/p&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwi_-dSOvuH8AhVBqpUCHT32Av8QguUECPIL&usg=AOvVaw1QXTHBbDi1PYbH9058evun
R$ 3.329,10 iPhone 12 64GB - Azul - Estou Zerado https://www.google.com.br/url?url=https://www.trocafy.com.br/iphon